<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/RL/ipynb/BERT_Base_Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except ModuleNotFoundError:
    IN_COLAB = False

Mounted at /content/drive


Check GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 16 17:38:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   44C    P0    43W / 250W |   1094MiB /  8192MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

ModuleNotFoundError: No module named 'psutil'

In [4]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git
!git pull

Cloning into 'Trade_With_Twitter'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (706/706), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 706 (delta 423), reused 447 (delta 215), pack-reused 0
Receiving objects: 100% (706/706), 176.72 KiB | 2.52 MiB/s, done.
Resolving deltas: 100% (423/423), done.
fatal: not a git repository (or any of the parent directories): .git


In [5]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [6]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
prices	rl  sentiment  trade.py  twitter  utils


In [1]:
!pip install -r ../requirements.txt

In [6]:
import os
import pandas as pd
from tqdm import tqdm

from sentiment.inference import get_tweets, get_sentiments, save_sentiments
from sentiment.folder import get_paths
from utils.dates import get_date_array, get_month_array, check_last_day

In [7]:
dates_range = ("2019-01-01", "2021-05-31")
#months_range = ["Jan 17", "Jun 21"]
# dates_range = ("2019-01-05", "2021-05-31")

#months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [8]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
model_name = "siebert/sentiment-roberta-large-english"

sentiment_analysis = pipeline("sentiment-analysis", model=model_name, device=0)

In [10]:
try:
    if IN_COLAB:
        source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData', model_name=model_name)
    else:
        source, results_folder = get_paths()
except:
    source, results_folder = get_paths()
source = os.path.normcase(source)
results_folder = os.path.normcase(results_folder)

results_folder

'..\\data\\bitcoin_scores\\distilbert-base-uncased-finetuned'

In [11]:
source, results_folder = "../data/bitcoin_tweets/", "../data/bitcoin_scores/sentiment-roberta-large-english_siebert"
source, results_folder =  os.path.normcase("../data/bitcoin_tweets/"),  os.path.normcase("../data/bitcoin_scores/sentiment-roberta-large-english_siebert")

source

'..\\data\\bitcoin_tweets\\'

In [ ]:
for date in dates:
    try:
        ids, tweets = get_tweets(date, source)

        percentage_per_chunk = 100
        save_every = 2000
        results = get_sentiments(date, tweets, ids, results_folder,
                                  sentiment_analysis=sentiment_analysis,
                                  save_every=save_every,
                                  percentage_per_chunk=percentage_per_chunk,
                                 slice_size=128,
                                 batch_size=100)
    except:
        print(f"{date} not found")
    # check_last_day(results_folder, date)

2019-01-01:  19%|██████████▋                                            | 28/144 [00:13<00:57,  2.02it/s]


2019-01-01 not found


2019-01-02: 100%|██████████████████████████████████████████████████████| 144/144 [01:06<00:00,  2.17it/s]


2019-01-02 not found


2019-01-03: 100%|██████████████████████████████████████████████████████| 144/144 [01:03<00:00,  2.27it/s]


2019-01-03 not found


2019-01-04: 100%|██████████████████████████████████████████████████████| 144/144 [01:04<00:00,  2.24it/s]


2019-01-04 not found
2019-01-05 not found


2019-01-06:  34%|██████████████████▋                                    | 49/144 [00:21<00:42,  2.23it/s]

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull